# Анализ текстов

## Введение

Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

## 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [23]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

## 2. Вычислите TF-IDF-признаки для всех текстов. 
Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
print(f"Lenght all data set: {len(newsgroups.data)}")
train_data_set = newsgroups.data[:1500]
test_data_set = newsgroups.data[1500:]

train_target = newsgroups.target[:1500]
test_target = newsgroups.target[1500:]
y = newsgroups.target

Lenght all data set: 1786


In [4]:
X = vectorizer.fit_transform(newsgroups.data)
print(type(X))
print(X.shape)
print(len(vectorizer.get_feature_names()))
print(X[1, 4000])
print(vectorizer.get_feature_names()[4000])

<class 'scipy.sparse.csr.csr_matrix'>
(1786, 28382)
28382
0.0
affairs


## 3. Подберите минимальный лучший параметр C.
Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy

In [5]:
grid = np.power(10.0, np.arange(-5, 6))
grid

array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])

In [6]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

In [7]:
for k, v in gs.cv_results_.items():
    print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [3.1987082  3.22697053 3.30407352 3.19902387 2.66859431 1.6336122
 1.62071385 1.61784368 1.61691251 1.62077379 1.62024078]

Key: std_fit_time, value: [0.04910825 0.06186094 0.03643364 0.05572188 0.01667668 0.02579374
 0.02839183 0.03287557 0.01879961 0.02517089 0.03466802]

Key: mean_score_time, value: [0.7668314  0.76048298 0.75466666 0.76246572 0.63974233 0.3780375
 0.38004193 0.37984109 0.38097835 0.37953    0.37835116]

Key: std_score_time, value: [0.00616827 0.00747841 0.00781807 0.00490738 0.00880687 0.00761874
 0.00612474 0.00937955 0.01135147 0.00880951 0.0100217 ]

Key: param_C, value: [1e-05 0.0001 0.001 0.01 0.1 1.0 10.0 100.0 1000.0 10000.0 100000.0]

Key: params, value: [{'C': 1e-05}, {'C': 0.0001}, {'C': 0.001}, {'C': 0.01}, {'C': 0.1}, {'C': 1.0}, {'C': 10.0}, {'C': 100.0}, {'C': 1000.0}, {'C': 10000.0}, {'C': 100000.0}]

Key: split0_test_score, value: [0.54469274 0.54469274 0.54469274 0.54469274 0.95810056 0.99441341
 0.99441341 0.99441341 0.9

## 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [8]:
print(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['param_C'])

[0.55263158 0.55263158 0.55263158 0.55263158 0.95016797 0.99328108
 0.99328108 0.99328108 0.99328108 0.99328108 0.99328108]
[1e-05 0.0001 0.001 0.01 0.1 1.0 10.0 100.0 1000.0 10000.0 100000.0]


In [9]:
print(np.argmax(gs.cv_results_['mean_test_score']))
print(gs.cv_results_['param_C'][5])

5
1.0


In [10]:
clf1 = SVC(kernel='linear', C=1.0, random_state=241)
clf1.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

# 5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [11]:
import sklearn
print(sklearn.__version__)
clf1.coef_

0.20.3


<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [51]:
coef = clf1.coef_.toarray()
coef.shape

(1, 28382)

In [52]:
coef1 = pd.Series(coef[0])
coef1.head()

0    0.292581
1   -0.123148
2    0.000000
3    0.000000
4    0.000000
dtype: float64

In [60]:
top_10 = abs(coef1).sort_values().tail(10)
indices=top_10.index
feature_mapping=vectorizer.get_feature_names()
words = []

In [61]:
for i in indices:
    words.append(feature_mapping[i])

print(sorted(words))

['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion', 'sci', 'sky', 'space']
